Preparation of plots for LARS Paper IV
===================================

*This time I will try and document whan I am doing really thrroughly, in the hope that it will one day become useful to someone out there.*

This notebook is mainly about creating nice looking plots for the paper. It relies on the data already being processed and saved in the formats specified in the python file `cospaper.py`, and relies heavily on the code used there.

### Prerequisites to run:

The following is required to run this notebook:

+ `numpy` and `scipy` (the former a subset of the latter)
+ `matplotlib`
+ `astropy`
+ `pandas`
+ The files `cospaper.py`, `mpl_tricks.py` and `adhoc_functions.py`.
+ The input variables file `lineslists.py`
+ The pickled dicts of velocity-based dataframes of the LARS galaxies 
  as created by running the `cospaper.py` code.


### Imports

First, import of modules and packages that we need everywhere.

In [2]:
import matplotlib
#matplotlib.use('Qt4Agg')
import cospaper as cosp
import scipy as sp
import scipy.interpolate as itp
import scipy.optimize as opt
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib import widgets as wdg
import mpl_tricks as mp
from pprint import pprint
from LARS_branding import lbrand
import wesanderson as wa
import brewer2mpl as btm
import DynVel as dv
import sys
#%pprint

/home/trive/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


### Setup of Matplotlib et al.

In [3]:
plt.rcdefaults()
plt.rc('text', usetex=True)
plt.rc('font', **{'family':'serif', 'serif':['Computer Modern Roman'], 
                  'sans-serif':'Helvetica'})
cf_fonts = {'family': 'serif',}
plt.rc('font', **cf_fonts)

## Import and prepare data

In [4]:
dfdict = pd.read_pickle('../Products/dfdict.pickle')
vflis = pd.read_pickle('../Products/vflis.pickle')
vfhis = pd.read_pickle('../Products/vfhis.pickle')
dfdict[1].z_value = 0.020434136

In [5]:
vfhis[1].Velocity.min()

-4169.7641470814342

## Other global definitions

In [6]:
LyaEmitters = ['01']#, '02', '05', '07', '14']
#LyaAbsorbers = ['04', '06']

## Grid of overing fraction plots

Here, we first create a figure with a single `Axes` object with no spines or ticks showing. This way, we are certain to always have the labels set nicely centered.

Afterwards, we do some ugly hacking to use the already existing function in `cospaper.py` for drawing these plots, but only harvesting one of the `Axes` objects coming out of it. This is easier, after all, than writing it from scratch and fine tuning all the parameters etc.

First, different parameters together in one place so they are easier to find:

In [7]:
phase_string = 'Neutral'
cf_color = 'red'  # cosp.paircolors[1]  # 'green'   # Dot color
cb_color = 'pink'  # cosp.paircolors[0]  # 'lightgreen'  # Bar color
species = 'Si_II'

Next, the general setup and layout of the figure and axes grid:

In [8]:
def prepare_plot(twin_ax=False):
    cf_fig, subplots = plt.subplots(5, 3, sharex=True, sharey=True)
    cf_fig.set_size_inches(7, 5.2)
    cf_fig.dpi = 160
    cf_fig.axes[-1].set_visible(False)
    cntnr = cf_fig.add_subplot(
        111, frame_on=False,  
        xticks=[], yticks=[], xticklabels=[], yticklabels=[], 
        xlabel=r'Velocity [km s$^{-1}$]', ylabel='Relative flux')
    cntnr.set_xlabel(r'Velocity [km s$^{-1}$]', fontsize=10, labelpad=13)
    cntnr.set_ylabel('Flux / continuum', fontsize=9, labelpad=20)
    cf_fig.subplots_adjust(left=.05, right=.98, bottom=.07, top=.98, wspace=.02, hspace=.03)
    if twin_ax:
        rcont = plt.twinx(cntnr)
        rcont.set_yticks([])
        rcont.set_yticklabels([1, 2, 3], visible=False)
        rcont.set_ylabel(
            '{} covering fraction'.format(phase_string), 
            color=cf_color, labelpad=30, fontsize=10)
    return cf_fig, subplots

## Covering fraction plots

Now, cycle through LARS ID numbers, create one plot per galaxy.
A litle hacky-wacky is done here (you may now cringe), but trust me, the alternative is worse. Yeah, I'm not too good to get a little un-pythonic if that saves me a heck of a lot of typing...

In [9]:
reload(cosp)
cf_fig, subplots = prepare_plot(twin_ax=True)
cf_fig.set_size_inches(7, 5.2)
subplots = subplots.flatten()
dummyfig, dummyax = plt.subplots(2)

for i in range(1):
    ax = subplots[i]
    data = vflis[i+1]
    data = data.loc[data.Velocity.between(-700, 700)]
    cosp.draw_coverfrac_axes(
        data, dummyax[0], dummyax[1], ax, 
        dotcolor=cf_color, 
        barcolor=cb_color, 
        legend=False, ms=3)
    dax = dummyax[1]
    del dummyax[0].lines[:]
    del dummyax[1].lines[:]
    ax.tick_params(labelsize=10)
    if i in [0, 3, 6, 9, 12]:
        ax.set_yticks([0, 0.5, 1, 1.5])
        ax.set_yticklabels([0.0, 0.5, 1.0, 1.5], fontsize=8)
    if i in [12, 13]:
        ax.set_xticks([-600, -300,  0, 300, 600])
        ax.set_xticklabels([-600, -300,  0, 300, 600], fontsize=8)
    if i in [2, 5, 8, 11]:
        twax = plt.twinx(ax)
        twax.set_ylim(1.2, -1)
        twax.set_yticks([0, 0.5, 1])
        twax.tick_params(color=cf_color, labelcolor=cf_color, 
                         labelsize=8)
    larsname = data.columns.name
    if larsname[-2:] in LyaEmitters:
        larsname += '*'
    bbox_props = dict(boxstyle="square, pad=0.4", fc="w", ec='lightgray', lw=.8)
    ax.annotate(larsname, (-630, 1.6), 
                fontsize=9, bbox=bbox_props)
    
subplots[-1].set_xlim(-700, 700)
subplots[-1].set_ylim(-.2, 2.)
cf_fig.subplots_adjust(left=.07, right=.92, bottom=.07, top=.98, 
                       hspace=.03, wspace=.02)
#cf_fig.savefig('EWASS/coverfracs.pdf', facecolor='none')
plt.show()

In [9]:
plt.get_backend()

u'Agg'

## Covering fraction plot grid with Ly$\alpha$ profiles

In [10]:
import matplotlib.gridspec as gs
reload(cosp)
saveit = False  # True#False
import wesanderson as wa
wes = [farve for farve in wa.colors['Darjeeling2'] if farve == wa.colors['Darjeeling2'][1]]*3

CF_intervals = {1: [-450, 300], 
                2: [-600, 300], 
                3: [-600, 300], 
                4: [-500, 250],
                5: [-500, 200],
                6: [-400, 400],
                7: [-400, 300],
                8: [-600, 200],
                9: [-600, 400],
                10: [-500, 400],
                11: [-600, 450],
                12: [-550, 300],
                13: [-550, 400],
                14: [-650, 100]}

#vflis = cosp.load_dfdict('vflis.pickle')
#dfdict  = cosp.load_dfdict('dfdict.pickle')

###==============================================================
#        FIRST PANEL
###==============================================================

fig = plt.figure(figsize=(3.5, 3), dpi=160)
cont = plt.subplot(111, frameon=False)
cont.tick_params(labelleft='off', labelbottom='off')
cont.set_xlabel(r'\rm Velocity [km/s]', labelpad=15)
cont.set_ylabel(r'\rm Relative flux', labelpad=25)
basegrid = gs.GridSpec(3, 2)
dfig, daxes = plt.subplots(2)
dax1 = daxes[0]
dax2 = daxes[1]
#refgrid()
subgrids = {}
for  i in range(1):
    vdata = vflis[i+1]
    #data = vdata[vdata.Velocity.between(-1300, 1300)]
    wdata = dfdict[i+1]
    z = wdata.z_value
    subgrids[i+1] = gs.GridSpecFromSubplotSpec(7, 1, subplot_spec=basegrid[(i%7-i%2)/2, i%2], 
                                               wspace=.02, hspace=.0) 
    ax1 = plt.Subplot(fig, subgrids[i+1][:4])
    ax1.tick_params(labelsize=9)
    ax1.label_outer()
    ax2 = plt.Subplot(fig, subgrids[i+1][4:], sharex=ax1)
    ax2.tick_params(labelsize=9)
    ax2.set_yticks([0, .5, 1.])
    ax2.axis((-850, 850, -.2, 1.6))
    [j.set_linewidth(.6) for j in ax1.spines.itervalues()]
    [j.set_linewidth(.6) for j in ax2.spines.itervalues()]
    
    fig.add_subplot(ax1); fig.add_subplot(ax2)
    
    if i%2 == 1:
        ax1.tick_params(labelleft='off', labelright='on')
        ax2.tick_params(labelleft='off', labelright='on')
        twax = plt.twinx(ax2)
        #twax.set_frame_on(True)
        #twax.spines['right'].set_visible(True)
        twax.set_ylim(1.2, -.6)
        twax.set_yticks([0, .5, 1])
        twax.set_ylabel(r'$f_c(v)$', color=cf_color)
        twax.tick_params(color=cf_color, labelcolor=cf_color, labelsize=9, length=3)
        #[j.set_linewidth(.6) for j in twax.spines.itervalues()]
        #ax1.set_yticklabels([], visible=False)
        ax2.set_yticklabels([], visible=False)
        #fig.add_subplot(twax)

    
    if i < 4:
        ax2.set_xticklabels([], visible=False)
    
    #print vdata
    #mport ipdb; ipdb.set_trace()  # XXX BREAKPOINT
    cosp.draw_coverfrac_axes(vdata, dax1, dax2, ax2, thin=.3, legend=False, ms=2., cf_lims=CF_intervals[i+1])
    
    # Lyman Alpha part:
    lyawave = 1215.668 * (1. + z)
    wdata = wdata[wdata['lambda'].between(lyawave-10, lyawave+10)]
    restwave = wdata['lambda'] / (1 + z)
    lyavels = (restwave - 1215.668) / 1215.668 * sp.constants.c / 1000
    interval = lyavels[(lyavels > -3000) & (lyavels < 3000)]
    #topval = wdata['flam'].loc[(lyavels > -1600) & (lyavels < 1600)].max() 
    topval2 = wdata['flam'].median()  # loc[(lyavels > -1600) & (lyavels < 1600)].max()
    topval = wdata['flam'].loc[(lyavels > -1600) & (lyavels < 1600)].max()/topval2
    topval = max(topval, 1.)
    #print lyavels.shape, wdata['flam'].shape
    #break
    ax1.plot(lyavels, wdata['flam']/topval2, lw=.7, color='.1', drawstyle='steps-mid')
    mp.fill_between_steps(lyavels.values,  # Hacky-hacky: with white first.
                          (wdata['flam'].values-wdata['stddev'].values)/topval2, 
                          (wdata['flam'].values+wdata['stddev'].values)/topval2, 
                          facecolor='w', edgecolor='w', ax=ax1, lw=.1)
    mp.fill_between_steps(lyavels.values, 
                          (wdata['flam'].values-wdata['stddev'].values)/topval2, 
                          (wdata['flam'].values+wdata['stddev'].values)/topval2, 
                          facecolor=wes[2], 
                          edgecolor='.3',#wes[0], 
                          ax=ax1, lw=.1, alpha=.4)
    ax1.axis((-1200, 1200, -.2*topval, 1.25*topval))
    ax1.axvline(x=0., lw=.5, ls='--', color='k', zorder=0)
    ax1.axhline(y=0., lw=.5, ls='--', color='k', zorder=0)
    ax1.yaxis.set_ticks(ax1.yaxis.get_ticklocs()[sp.where(ax1.yaxis.get_ticklocs() >= 0.)])
    larsname = r'\rm '+ wdata.columns.name
    #if larsname[-2:] in LyaEmitters:
    #    larsname += '*'
    #elif larsname[-2:] in LyaAbsorbers:
    #    larsname += '$\dagger$'
    bbox_props = dict(boxstyle="square, pad=0.4", fc="w", ec='gray', lw=.5)
    ax1.annotate(larsname, xy=(.05, .83), xycoords='axes fraction', size=9, )
                # bbox=bbox_props)
        
fig.subplots_adjust(bottom=.06, top=.95, hspace=.02, wspace=.02, left=.08, right=.93)
if saveit: fig.savefig('../Figs/CoFraLyaH11.pdf', dpi=300)
#if saveit: fig.savefig('/home/trive/Desktop/CoFraLya13-14.pdf', dpi=300)
plt.show(fig)


### Same as above, but with lowest Fc's only.

In [11]:
import matplotlib.gridspec as gs
reload(cosp)

import wesanderson as wa
wes = [farve for farve in wa.colors['Darjeeling2'] if farve != wa.colors['Darjeeling2'][0]]

CF_intervals = {1: [-450, 300], 
                2: [-600, 300], 
                3: [-600, 300], 
                4: [-500, 250],
                5: [-500, 200],
                6: [-400, 400],
                7: [-400, 300],
                8: [-600, 200],
                9: [-600, 400],
                10: [-500, 400],
                11: [-600, 450],
                12: [-550, 300],
                13: [-550, 400],
                14: [-650, 100]}

vflis = cosp.load_dfdict('vflis.pickle')
dfdict  = cosp.load_dfdict('dfdict.pickle')

fig = plt.figure(figsize=(7, 5), dpi=160)
cont = plt.subplot(111, frameon=False)
cont.tick_params(labelleft='off', labelbottom='off')
cont.set_xlabel(r'\rm Velocity [km/s]', labelpad=15)
cont.set_ylabel(r'\rm Relative flux', labelpad=25)
basegrid = gs.GridSpec(2, 2)
dfig, daxes = plt.subplots(2)
dax1 = daxes[0]
dax2 = daxes[1]
#refgrid()
subgrids = {}
for  i, gal in enumerate([1, 7, 13, 14]):  # range(2):
    vdata = vflis[gal]
    wdata = dfdict[gal]
    subgrids[i+1] = gs.GridSpecFromSubplotSpec(7, 1, subplot_spec=basegrid[(i%7-i%2)/2, i%2], 
                                               wspace=.02, hspace=.0) 
    ax1 = plt.Subplot(fig, subgrids[i+1][:4])
    ax1.tick_params(labelsize=9)
    ax1.label_outer()
    ax2 = plt.Subplot(fig, subgrids[i+1][4:], sharex=ax1)
    ax2.tick_params(labelsize=9)
    ax2.set_yticks([0, .5, 1.])
    ax2.axis((-850, 850, -.2, 1.6))
    [j.set_linewidth(.6) for j in ax1.spines.itervalues()]
    [j.set_linewidth(.6) for j in ax2.spines.itervalues()]
    
    fig.add_subplot(ax1); fig.add_subplot(ax2)
    
    if i%2 == 1:
        ax1.tick_params(labelleft='off', labelright='on')
        ax2.tick_params(labelleft='off')
        twax = plt.twinx(ax2)
        twax.set_ylim(1.2, -.6)
        twax.set_yticks([0, 0.5, 1])
        twax.set_ylabel(r'$f_c(v)$', color=cf_color)
        twax.tick_params(color=cf_color, labelcolor=cf_color, 
                         labelsize=9, length=3)
    
    #if i%2 == 1:
    #    ax1.tick_params(labelleft='off', labelright='on')
    #    ax2.tick_params(labelleft='off', labelright='on')
        #ax1.set_yticklabels([], visible=False)
        ax2.set_yticklabels([], visible=False)

    #fig.add_subplot(ax1); fig.add_subplot(ax2)
    
    if i < 0:
        ax2.set_xticklabels([], visible=False)
    
    cosp.draw_coverfrac_axes(vdata, dax1, dax2, ax2, thin=.3, legend=False, ms=2., cf_lims=CF_intervals[gal])
    
    # Lyman Alpha part:
    restwave = wdata['lambda'] / (1 + wdata.z_value)
    lyavels = (restwave - 1215.668) / 1215.668 * sp.constants.c / 1000
    interval = lyavels[(lyavels > -3000) & (lyavels < 3000)]
    
    topval2 = wdata['flam'].median()  # loc[(lyavels > -1600) & (lyavels < 1600)].max() 
    topval = wdata['flam'].loc[(lyavels > -1600) & (lyavels < 1600)].max()/topval2
    topval = max(topval, 1.)
    ax1.plot(lyavels, wdata['flam']/topval2, lw=.7, color='.1', drawstyle='steps-mid')
    mp.fill_between_steps(lyavels.values,  # Hacky-hacky: with white first.
                          (wdata['flam'].values-wdata['stddev'].values)/topval2, 
                          (wdata['flam'].values+wdata['stddev'].values)/topval2, 
                          facecolor='w', edgecolor='w', ax=ax1, lw=.1)
    mp.fill_between_steps(lyavels.values, 
                          (wdata['flam'].values-wdata['stddev'].values)/topval2, 
                          (wdata['flam'].values+wdata['stddev'].values)/topval2, 
                          facecolor=wes[2], 
                          edgecolor='darkgray', #wes[0], 
                          ax=ax1, lw=.1, alpha=.99)
    ax1.axis((-1200, 1200, -.2*topval, 1.25*topval))
    ax1.axvline(x=0., lw=.5, ls='--', color='k', zorder=0)
    ax1.axhline(y=0., lw=.5, ls='--', color='k', zorder=0)
    ax1.yaxis.set_ticks(ax1.yaxis.get_ticklocs()[sp.where(ax1.yaxis.get_ticklocs() >= 0.)])
    larsname = r'\rm '+ wdata.columns.name
    if larsname[-2:] in LyaEmitters:
        larsname += '*'
    bbox_props = dict(boxstyle="square, pad=0.4", fc="w", ec='gray', lw=.5)
    ax1.annotate(larsname, xy=(.05, .83), xycoords='axes fraction', size=9, 
                 bbox=bbox_props)
        
fig.subplots_adjust(bottom=.13, top=.87, hspace=.02, wspace=.02, left=.08, right=.93)
fig.savefig('EWASS/CoFraLyaLowCF.pdf', facecolor='none')

plt.show()



## Ly$\alpha$ and LIS profiles for the broad, deep absorbers

In [6]:
reload(cosp)
#%matplotlib inline
#%matplotlib qt

import wesanderson as wa
wes = [farve for farve in wa.colors['Darjeeling2']]# if farve != wa.colors['Darjeeling2'][0]]

#vflis = cosp.load_dfdict('vflis.pickle')
#dfdict  = cosp.load_dfdict('dfdict.pickle')

fig = plt.figure(figsize=(3.5, 4.5), dpi=160)
cont = plt.subplot(111, frameon=False)
cont.tick_params(labelleft='off', labelbottom='off', length=0, labelsize=9)
cont.set_xlabel(r'\rm Velocity [\AA]', labelpad=15)
cont.set_ylabel(r'\rm Normalised flux', labelpad=20)

for i, gal in enumerate([6, 9, 10]):
    ax = fig.add_subplot(3, 1, i+1)
    ax.label_outer()
    ax.tick_params(labelsize=9, length=2)
    ax.yaxis.set_ticks([0, .2, .4, .6, .8, 1., 1.2, 1.4])
    [j.set_linewidth(.6) for j in ax.spines.itervalues()]
    wdata = dfdict[gal]
    vdata = vflis[gal]
    # Lyman Alpha part:
    restwave = wdata['lambda'] / (1 + wdata.z_value)
    lyavels = (restwave - 1215.668) / 1215.668 * sp.constants.c / 1000
    interval = lyavels[(lyavels > -3000) & (lyavels < 3000)]
    topval1 = wdata['flam'].mean() 
    topval2 = wdata['flam'].median()
    topval = topval2  #sp.mean([topval1, topval2])
    ax.plot(lyavels, wdata.smooth/topval, lw=.5, color=wes[1], drawstyle='steps-mid')
    ax.axis((-6500, 6500, -.2, 1.8))
    lisidx = vdata.Velocity[vdata.Velocity.abs() < 2000].index
    ax.plot(vdata.Velocity[lisidx], vdata.MeanProfile[lisidx], lw=.5, color=wes[4], 
            drawstyle='steps-mid')#wes[2])
    ax.axhline(y=0, ls=':', lw=.5, color='k')
    ax.axvline(x=0, ls=':', lw=.5, color='k')
    larsname = r'\rm '+ wdata.columns.name
    if gal == 6:
        larsname += '$\dagger$'
    bbox_props = dict(boxstyle="square, pad=0.4", fc="w", ec='lightgray', lw=.5)
    ax.annotate(larsname, xy=(.05, .85), xycoords='axes fraction', size=9,) 
                 #bbox=bbox_props)
    
fig.subplots_adjust(hspace=.04, left=.13, right=.95, bottom=.1, top=.95)
#plt.savefig('Paper/Figs/broadlines.pdf', dpi=300, facecolor='none')
fig.set_size_inches((5, 4))
#plt.savefig('EWASS/broadlines.pdf', dpi=300, facecolor='none')
fig.set_size_inches((3.4, 4.5))
#fig.dpi=300
#display.display(fig)
plt.show()

KeyError: 6

## Make a single triple-panel covering fraction plot:

In [16]:
reload(cosp)
#vflis = cosp.load_dfdict('vflis.pickle')
cf_color = cosp.paircolors[3]  # 'green'   # Dot color
cb_color = cosp.paircolors[2]  # 'lightgreen'  # Bar color
fig = plt.figure(figsize=(3.5, 3.5), dpi=160)
ax1 = plt.subplot(411)
ax2 = plt.subplot(412, sharex=ax1)
ax3 = plt.subplot(212, sharex=ax1)
ax4 = plt.twinx(ax3)
[i.set_linewidth(.5) for i in ax1.spines.itervalues()]
[i.set_linewidth(.5) for i in ax2.spines.itervalues()]
[i.set_linewidth(.5) for i in ax3.spines.itervalues()]
[i.set_linewidth(.5) for i in ax4.spines.itervalues()]

for ax in [ax1, ax2, ax3]:
    ax.label_outer()
    ax.tick_params(labelsize=9)
fig.subplots_adjust(hspace=0, bottom=.11, right=.88)
cosp.draw_coverfrac_axes(vflis[1], ax1,  ax2, ax3, thick=1.2, thin=.4, 
                         dotcolor=cf_color, barcolor=cb_color)
ax1.axis((-600, 600, -5, 80))
ax1.set_yticks([0, 20, 40, 60, 80])
ax1.set_ylabel(r'$\chi^2 + 1$', fontsize=10)
ax2.axis((-600, 600, 8, 16))
ax2.set_yticks([8, 10, 12, 14])
ax2.set_ylabel('log $N_{\mathrm{SiII}}$', fontsize=10)
ax3.axis((-600, 600, -.2, 1.6))
ax3.set_yticks([0, .2, .4, .6, .8, 1.])
ax3.set_ylabel('Relative flux', labelpad=3, fontsize=10)
ax3.set_xlabel('Velocity [km/s]', labelpad=3, fontsize=10)
ax4.axis((-600, 600, 1.2, -.6))
ax4.tick_params(color=cf_color, labelcolor=cf_color, labelsize=9)
ax4.set_yticks([0, .2, .4, .6, .8, 1.])
ax4.set_ylabel('$f_c(v)$', color=cf_color, fontsize=10)
plt.xlim(-900, 600)
#fig.savefig('Figures/LARS_01_Si_II.pdf')
#fig.savefig('Paper/Figs/LARS_01_Si_II.pdf')
#fig.savefig('EWASS/CoFraL1.pdf',facecolor='none')
plt.show()

## Full spectrum plots

In [7]:
dfd = pd.read_pickle('../Products/dfdict.pickle')
dfd[1].z_value = 0.020434136
import matplotlib.patheffects as PathEffects
import wesanderson as wa
#import viztools as viz
wes = [farve for farve in wa.colors['Darjeeling2'] if farve != wa.colors['Darjeeling2'][0]]
wes = cosp.paircolors
from linelists import wlsdict
wlsdict[r'Ly $\alpha$'] = 1215.67
plt.rc('font', **{'family':'serif', 'serif':['Computer Modern Roman'], 
                  'family':'sans-serif', 'sans-serif':'Helvetica'})
plt.rc('text', usetex=True)

def set_lines(redshift=0, ax=None, ann=False, MW=True):
    ''' 
    Assuming wavelengths are converted to rest frame, so MW lines are blueshifted
    '''
    if ax is None:
        ax = plt.gca()
    for line in wlsdict.keys():
        species = ' '.join(line.split()[:2])
        #  don't let line annotations overlap:
        wavl = wlsdict[line]
        if (sp.floor(wavl) == 1190) or (species == 'O I'):
            hal = 'right'
        elif (sp.floor(wavl) == 1193) or (sp.floor(wavl) == 1304):
            hal = 'left'
        else:
            hal = 'center'
        ax.axvline(x=wlsdict[line], lw=.6, color=wes[1], zorder=0, alpha=.99)
        if ann and ((wlsdict[line] > 1125) & (wlsdict[line] < 1600)):
            ax.annotate(species, (wlsdict[line], 1.05), xycoords=('data', 'axes fraction'), 
                        size=6, color=wes[1], name='sans-serif', rotation='vertical', 
                        ha=hal, va='bottom')
        if MW:
            if ((wlsdict[line]/(1+redshift) > max(wave.min(), 1125.)) & 
                (wlsdict[line]/(1+redshift) < min(wave.max(), 1600.))):
                ax.annotate(species, xy=(wlsdict[line]/(1.+redshift), 0.06),  xycoords=('data', 'axes fraction'),
                            size=6, color=wes[7], family='monospace', rotation='vertical', 
                            ha=hal, va='bottom', 
                            path_effects=[PathEffects.withStroke(linewidth=3.5, foreground="w")])
                ax.axvline(x=wlsdict[line]/(1+redshift), ymin=.9, lw=2.4, color='w', zorder=2,)  
                ax.axvline(x=wlsdict[line]/(1+redshift), ymin=.9, lw=0.8, color=wes[7], zorder=3,)    

# Define container Axes to do axes labels an' stuff
fig = plt.figure(dpi=160, figsize=(7, 2.))
cont = fig.add_subplot(111, frameon=False)
plt.setp(cont.get_yticklabels(), visible=False)
plt.setp(cont.get_xticklabels(), visible=False)
cont.set_xlabel(r'\rm Rest-frame wavelength [\AA]', fontsize=9)
cont.set_ylabel(r'\rm Flux [ $\mathrm{10^{-15}\, erg\, cm^{-2}\, s^{-1}\, \AA^{-1}}$ ]', 
                fontsize=9)
cont.yaxis.labelpad = 20
cont.xaxis.labelpad = 15
#viz.prett
# First axes is special, sos gets special treatment.
data = dfd[1]
ax1 = fig.add_subplot(1, 1, 1)
[i.set_linewidth(0.5) for i in ax1.spines.itervalues()] 
ax1.label_outer()
wave = data['lambda'] / (1 + data.z_value)
flux = data['smooth'] * 1e15 / 6.
plt.plot(wave, flux, color='w', lw=3.5)
plt.plot(wave, flux, color='k', lw=0.3)
top = flux.loc[data['lambda'].between(1350,  1400)].median() * 4
ax1.set_ybound(lower=top*-.30, upper=top)
ax1.tick_params(labelsize=8, length=2)
#ax1.yaxis.set_major_locator(plt.MaxNLocator(5))
larsname = data.columns.name
ax1.annotate(larsname, (.92, .1), xycoords='axes fraction', fontsize=7)
set_lines(ax=ax1, redshift=data.z_value, ann=True)
ax1.set_xbound(lower=1125, upper=1570)
ax1.axhline(0, color='k', ls='--', lw=.6)
plt.subplots_adjust(left=0.08, bottom=0.2, right=.98, top=.85)
fig.savefig('../Figs/FullSpec.pdf')
fig.savefig('../Figs/FullSpec.png', dpi=300)
#fig1.savefig('Paper/Figs/FullSpec8-14.pdf', dpi=300)
#fig1.savefig('EWASS/FullSpec8-14.pdf', dpi=300, facecolor='none')
# print [' '.join(l.split()[:2]) for l in wlsdict.iterkeys()]
plt.show()

In [164]:
dfd = cosp.load_dfdict('dfdict.pickle')
wave8, flux8 = dfd[8]['lambda'], dfd[8]['smooth']
plt.plot(wave8, flux8)
print dfd[8].columns.name
wave7, flux7 = dfd[7]['lambda'], dfd[7]['smooth']
plt.plot(wave7, flux7)
print dfd[7].columns.name
plt.show()

LARS 08
LARS 07


## Compare phases

Now, by popular demand, a plot comparing the average profiles of neutral and ionized gas, same style I did it for Tololo 1247.

In [28]:
ph_fig, subplots = prepare_plot()
subplots = subplots.flatten()
moonrise2 = [farve for farve in wa.colors['Darjeeling2'] if farve != wa.colors['Darjeeling2'][0]]
#moonrise2 = [farve for farve in wa.colors['Moonrise2']]
for i in range(14):
    ax = subplots[i]
    [k.set_linewidth(.6) for k in ax.spines.itervalues()]
    j = i + 1
    # Naming for convenience:
    lisvels = vflis[j].Velocity
    lismean = vflis[j].MeanProfile
    liserrs = vflis[j].MeanErrors
    if j in vfhis.keys():  # Because LARS 11 sucks
        hisvels = vfhis[j].Velocity
        hismean = vfhis[j].MeanProfile
        hiserrs = vfhis[j].MeanErrors

    lismean = sp.convolve(lismean, sp.ones(5)/5., mode='same')
    liserrs = sp.convolve(liserrs, sp.ones(5)/5., mode='same')
    hismean = sp.convolve(hismean, sp.ones(5)/5., mode='same')
    hiserrs = sp.convolve(hiserrs, sp.ones(5)/5., mode='same')

    if j in vfhis.keys():  # Because LARS 11 sucks
        ax.plot(hisvels, hismean,
                label=r'\rm Mean Si IV profile', drawstyle='steps-mid', 
                color=moonrise2[3], lw=1., zorder=1)
        mp.fill_between_steps(hisvels.values, hismean+hiserrs, hismean-hiserrs, 
                              ax=ax,
                              color='w', alpha=.5, lw=.2, 
                              edgecolor='w', zorder=0)
        mp.fill_between_steps(hisvels.values, hismean+hiserrs, hismean-hiserrs, 
                              ax=ax,
                              color=moonrise2[0], alpha=.5, lw=.2, 
                              edgecolor=moonrise2[3], zorder=0)
    ax.plot(lisvels, lismean, #vflisTololo[1].MeanProfile, 
                label=r'\rm Mean LIS profile', drawstyle='steps-mid', 
            color=cosp.paircolors[11],
            #color=moonrise2[1],
            lw=1., zorder=4)
    mp.fill_between_steps(lisvels.values, lismean+liserrs, lismean-liserrs,
                          ax=ax,
                          color=cosp.paircolors[10], alpha=.4, lw=.2,
                          edgecolor=cosp.paircolors[11], zorder=3)
    if i in [0, 3, 6, 9, 12]:
        ax.set_yticks([0., 0.5, 1, 1.5])
        ax.set_yticklabels([0.0, 0.5, 1.0, 1.5], fontsize=8, family='serif')
    if i in [12, 13]:
        ax.set_xticks([-600, -300,  0, 300, 600])
        ax.set_xticklabels([-600, -300,  0, 300, 600], fontsize=8, family='serif')
    
    ax.axhline(y=1., lw=.5, color='k', ls='--', zorder=0)
    ax.axhline(y=0., lw=.5, color='k', ls='-', zorder=0 )
    ax.axvline(x=0., lw=.5, color='k', ls='--', zorder=0)
    larsname = r'\rm '+vflis[j].columns.name
    if larsname[-2:] in LyaEmitters:
        larsname += '*'
    elif larsname[-2:] in LyaAbsorbers:
        larsname += '$\dagger$'
    bbox_props = dict(boxstyle="square, pad=0.4", fc="w", ec='lightgray', lw=.8)
    ax.annotate(larsname, (-630, 1.6), 
                fontsize=10, family='serif', bbox=bbox_props)
    
subplots[-1].set_xlim(-700, 700)
subplots[-1].set_ylim(-.2, 2.)
ph_fig.subplots_adjust(left=.07, right=.95, bottom=.07, top=.98, 
                       hspace=.03, wspace=.02)
subplots[13].legend(
    loc='center', frameon=False, fontsize=11, 
    bbox_to_anchor=(1.5, 0.5), labelspacing=.3)

plt.savefig('Paper/Figs/comparePhases.pdf')
plt.show()

## Computations of FWHM of profiles

This one is a bit tougher. I want to get in a completely generic line profiles and measure their FWHM as is, no modelling or anything. Let's see how that goes.


In [73]:
def onselect(vmin, vmax):
    theidx = df[(veloc > vmin) & (veloc < vmax)].index
    ax.axvspan(vmin, vmax, color='orange', zorder=0)
    idxlist.append(theidx.values.tolist())
    fig.canvas.draw()
    
bigfig, subplots = prepare_plot()
subplots = subplots.flatten()
#plt.show(block=False)
FWHMdict = {} 

#def line_select():

for i in vflis.keys():
    df = vflis[i]
    avpro = df.MeanProfile
    averr = df.MeanErrors
    veloc = df.Velocity
    idxlist = []
    #line_select()
    #fig, ax = plt.subplots(1, 1)
    #ax.axis((-800, 800, -.2, 2.))
    #ax.axvline(x=0, color='k', ls='--', lw=1.)
    #ax.axhline(y=0, color='k', lw=1.)
    #ax.axhline(y=1., color='k', ls='--', lw=1.)
    #ax.plot(veloc, avpro)
    #Sel = wdg.SpanSelector(ax, onselect, 'horizontal', minspan=3.)
    #plt.show()

    print 'Now calculating...'
    # For later `onclick` function:
    idxlist = df[(-500 < df.Velocity) & (df.Velocity < 400)].index.values.tolist()
    #idxlist = [x for y in idxlist for x in y]
    subdf = df.loc[idxlist]
    minval = avpro.loc[idxlist].min()
    halfmax = 1 - (1 - minval) / 2.
    subvel = veloc.loc[idxlist].values
    subavp = avpro.loc[idxlist].values
    
    # Now, define the data as interpolation:
    cont = sp.ones(len(idxlist)) * halfmax
    contpolyn = itp.PiecewisePolynomial(subvel, cont[:, sp.newaxis])
    linepolyn = itp.PiecewisePolynomial(subvel, subavp[:, sp.newaxis])

    evalvels = sp.linspace(subvel.min(), subvel.max(), 1000)


    def pdiff(x):
        return linepolyn(x) - contpolyn(x)
    velmin = subvel.min()
    velmax = subvel.max()
    velmid = subvel[:-1] + sp.diff(subvel) / 2
    roots = set()
    
    for val in velmid:
        root, infodict, ier, mesg = opt.fsolve(pdiff, val, full_output=True)
        if ier == 1 and velmin < root < velmax:
            roots.add(root[0])
    roots = sp.array(list(roots))
    minout = roots.min()
    maxout = roots.max()
    FWHM = maxout - minout
    print minout, maxout, FWHM

    # For debugging!
    #fig, ax = plt.subplots(1,1)
    ax = subplots[i-1]
    ax.plot(veloc, avpro, 'k', drawstyle='steps-mid', lw=1.2)
    ax.axis((-700, 700, -.2, 2.))
    ax.plot(evalvels, linepolyn(evalvels), 'g', lw=.8)
    ax.axhline(y=0, color='k', ls='-', lw=1.)
    ax.axhline(y=1, color='k', ls='--', lw=1.)
    ax.axvline(x=0, color='k', ls='--', lw=1.)
    ax.axvline(x=minout, color='gray', ls=':', lw=.8)
    ax.axvline(x=maxout, color='gray', ls=':', lw=.8)
    ax.axhline(y=halfmax, color='gray', ls=':', lw=.8)
    ax.annotate('FWHM: {0:.2f}'.format(FWHM), ((minout + maxout)/2, 1.5), 
                backgroundcolor='w', ha='center', va='bottom',  size='small')
    ax.annotate(df.columns.name, (650, 0.1), 
                backgroundcolor='w', ha='right', va='bottom')
    #ax.plot((minout, maxout), (halfmax, halfmax), 
    #        color='orange', ls='--', lw=1.5)
    ax.plot((minout, maxout), (1.3, 1.3), 'k|-', mew=1., ms=3)
    FWHMdict[i] = {'Minvel': minout, 'Maxvel': maxout, 'FWHM': FWHM} 

bigfig.subplots_adjust(left=.1, right=.95, bottom=.1, top=.95, 
                       hspace=.04, wspace=.02)
plt.show()

Now calculating...
-241.962736743 55.1369014468 297.09963819
Now calculating...
-158.871269802 49.7893937742 208.660663576
Now calculating...
-363.660514672 117.026382556 480.686897229
Now calculating...
-172.723310857 90.922521645 263.645832502
Now calculating...
-346.614456058 18.829530078 365.443986136
Now calculating...
-226.723422101 316.843614895 543.567036996
Now calculating...
-154.191783043 85.8633246305 240.055107674
Now calculating...
-435.860691238 -46.9759356415 388.884755597
Now calculating...
-221.943682875 195.955067208 417.898750083
Now calculating...
-218.85282288 154.094737937 372.947560817
Now calculating...
-386.525087415 -8.95956772295 377.565519692
Now calculating...
-227.381928769 204.364397509 431.746326277
Now calculating...
-304.564324514 116.402075442 420.966399956
Now calculating...
-385.188010066 11.9021397 397.090149766


### Second try, now with LLE instead of interpolation, and with Monte Carlo!

In [82]:
import statsmodels.nonparametric.kernel_regression as krg

def prettify(ax):
    ax.tick_params(length=2, labelsize=9)
    [j.set_linewidth(.6) for j in ax.spines.itervalues()]
    ax.axhline(0, color='k', ls='--', lw=.5)
    ax.axvline(0, color='k', ls='--', lw=.5)
    return ax

def onselect(vmin, vmax):
    theidx = df[(veloc > vmin) & (veloc < vmax)].index
    ax.axvspan(vmin, vmax, color='orange', zorder=0)
    idxlist.append(theidx.values.tolist())
    fig.canvas.draw()
    
def pdiff(x):
    return linepolyn(x) - contpolyn(x)
    
bigfig, subaxes = prepare_plot()
subaxes = subaxes.flatten()
# Uncomment to redo calculations:
FWHMdict = {} 

for i in vflis.keys():
    df = vflis[i]
    print df.columns.name
    avpro = df.MeanProfile
    averr = df.MeanErrors
    veloc = df.Velocity
    idxlist = []

    print 'Now calculating...'
    # For later `onclick` function:
    idxlist = df[(-500 < df.Velocity) & (df.Velocity < 400)].index.values.tolist()
    subdf = df.loc[idxlist]
    minval = avpro.loc[idxlist].min()
    halfmax = 1 - (1 - minval) / 2.
    cont = halfmax
    subvel = veloc.loc[idxlist].values
    subavp_orig = avpro.loc[idxlist].values
    errmean = averr.mean()
    errspread = averr.std()
    FWHMlist = []
    count = 0
    MLE = krg.KernelReg(subavp_orig, subvel, var_type='c', bw='cv_ls')#[20])#bw='aic')
    kernel_width = MLE.bw[0]

    while count < 100:  # Make 'while' instead
    #while 1 == 2:
        signs = sp.around(sp.random.random(len(subavp_orig)))
        signs[signs == 0.] = -1.
        subavp = subavp_orig + sp.random.normal(errmean, errspread, len(subavp_orig)) * signs
        minval = subavp.min()
        halfmax = max(1 - (1 - minval) / 2., .5)

        mle = krg.KernelReg(subavp, subvel, var_type='c', bw=[kernel_width])
        means, mfx = mle.fit()

        spl = itp.UnivariateSpline(subvel, (means - halfmax), k=3)
        roots = spl.roots()

        if len(roots) > 1:
            FWHM = roots.max() - roots.min()
            FWHMlist.append(FWHM)
            count += 1
        else:
            FWHM = sp.nan
 
        sys.stdout.write("\r Iteration {}, FWHM: {}".format(count, FWHM))
        sys.stdout.flush()
 
    MLE = krg.KernelReg(subavp_orig, subvel, var_type='c', bw='cv_ls')#[20])#bw='aic')
    means, mfx = MLE.fit()
    spl = itp.UnivariateSpline(subvel, means-cont)
    FWHM = spl.roots().max() - spl.roots().min()#ptp()
    print ''
    #print FWHM
    kernelwidth = mle.bw[0]

    ax = subaxes[i-1]
    subaxes[i-1].plot(veloc, avpro, lw=1.2, color='k', drawstyle='steps-mid')
    subaxes[i-1].plot(subvel, means, color='orange')
    subaxes[i-1].axis((-600, 600, -means.min()*.1, means.max()*1.6))
    ax.axhline(y=0, color='k', ls='-', lw=.5)
    ax.axhline(y=1, color='k', ls='--', lw=.5)
    ax.axvline(x=0, color='k', ls='--', lw=.5)
    ax.axhline(y=cont, color='b', lw=.2)
    ax = prettify(ax)
    FWHMarr = sp.array(FWHMlist)
    print FWHM, FWHMarr.mean(), FWHMarr.std()
    print '\n'
    #print FWHMarr.std()
    # Uncomment to redo calculations:
    FWHMdict[i] = {'FWHM': FWHMarr.mean(), 'Error': FWHMarr.std(), 'Count': count} 

bigfig.subplots_adjust(left=.1, right=.95, bottom=.1, top=.95, 
                       hspace=.04, wspace=.02)
bigfig.show()

LARS 01
Now calculating...
 Iteration 100, FWHM: 259.41205866
333.447892823 269.762741405 32.7329266951


LARS 02
Now calculating...
 Iteration 100, FWHM: 249.087644526
153.520058178 168.460736596 65.5012977805


LARS 03
Now calculating...
 Iteration 100, FWHM: 510.264521388
528.363500488 516.954019708 22.3849577495


LARS 04
Now calculating...
 Iteration 100, FWHM: 362.996742921
315.367960348 310.869786421 41.7742303613


LARS 05
Now calculating...
 Iteration 100, FWHM: 331.114388656
391.776416042 346.523004946 16.993695183


LARS 06
Now calculating...
 Iteration 100, FWHM: 650.617488294
290.348641461 474.668619565 196.94711241


LARS 07
Now calculating...
 Iteration 100, FWHM: 222.364708459
292.7351151 153.125377967 59.0909294022


LARS 08
Now calculating...
 Iteration 100, FWHM: 416.062230727
412.697249908 409.964341601 64.6086660362


LARS 09
Now calculating...
 Iteration 100, FWHM: 453.25810175
466.914089042 462.777059821 18.6094108612


LARS 10
Now calculating...
 Iteration 100, 

In [83]:
plt.show()
#FWHMdict

In [84]:
FWHMmcdf = pd.DataFrame.from_dict(FWHMdict).T
FWHMmcdf.to_csv('FWHM-LLE-MC.csv')
FWHMmcdf

,Count,Error,FWHM
1,100,32.732927,269.762741
2,100,65.501298,168.460737
3,100,22.384958,516.954020
4,100,41.774230,310.869786
5,100,16.993695,346.523005
6,100,196.947112,474.668620
7,100,59.090929,153.125378
8,100,64.608666,409.964342
9,100,18.609411,462.777060
10,100,26.989336,445.892427


### LIS vs. HI width

In [74]:
FWHMdf = pd.DataFrame.from_dict(FWHMdict).T
FWHMdf#ict.pop('HIWidths')

,FWHM,Maxvel,Minvel
1,297.099638,55.136901,-241.962737
2,208.660664,49.789394,-158.871270
3,480.686897,117.026383,-363.660515
4,263.645833,90.922522,-172.723311
5,365.443986,18.829530,-346.614456
6,543.567037,316.843615,-226.723422
7,240.055108,85.863325,-154.191783
8,388.884756,-46.975936,-435.860691
9,417.898750,195.955067,-221.943683
10,372.947561,154.094738,-218.852823


In [9]:
FWHMdf.to_csv('FWHMs.csv')

In [102]:
FWHMmcdf = pd.read_csv('FWHM-LLE-MC.csv', index_col=0)
#FWHMmcdf
spardywidths = [160, 140, 170, 180, 160, 370, 100, 
                240, 310, 280, 260, sp.nan, sp.nan, sp.nan]
spardyerrs = [10, 4, 20, 4, 28, 3.3, 32, 4, 18, 140, 11, sp.nan, sp.nan, sp.nan]
FWHMmcdf['HIWidths'] = spardywidths
FWHMmcdf['HIErrs'] = spardyerrs
FWHMmcdf.columns =  ['Iter', 'LIS Errs', 'FWHM(LIS)', 'FWHM (HI)', 'HI Errs']
FWHMmcdf

,Count,Error,FWHM,HIWidths,HIErrs
1,100,32.732927,269.762741,160,10.0
2,100,65.501298,168.460737,140,4.0
3,100,22.384958,516.954020,170,20.0
4,100,41.774230,310.869786,180,4.0
5,100,16.993695,346.523005,160,28.0
6,100,196.947112,474.668620,370,3.3
7,100,59.090929,153.125378,100,32.0
8,100,64.608666,409.964342,240,4.0
9,100,18.609411,462.777060,310,18.0
10,100,26.989336,445.892427,280,140.0


In [103]:
print FWHMmcdf.to_string()

    Count       Error        FWHM  HIWidths  HIErrs
1     100   32.732927  269.762741       160    10.0
2     100   65.501298  168.460737       140     4.0
3     100   22.384958  516.954020       170    20.0
4     100   41.774230  310.869786       180     4.0
5     100   16.993695  346.523005       160    28.0
6     100  196.947112  474.668620       370     3.3
7     100   59.090929  153.125378       100    32.0
8     100   64.608666  409.964342       240     4.0
9     100   18.609411  462.777060       310    18.0
10    100   26.989336  445.892427       280   140.0
11    100  184.214935  517.351992       260    11.0
12    100   26.208175  374.830791       NaN     NaN
13    100   52.459364  372.911215       NaN     NaN
14    100   59.390792  128.696088       NaN     NaN


### FWHM comparison plot:

In [101]:
import matplotlib.font_manager as fm
fig, ax = plt.subplots(1, 1)
ax.tick_params(labelsize=8)
fig.set_size_inches(3.4, 3.4)
fig.set_dpi(160)
[j.set_linewidth(.6) for j in ax.spines.itervalues()]

for i in FWHMdict.keys():
    if i > 11:
        continue
    SiIIwidth = FWHMmcdf.loc[i]['FWHM(LIS)']
    SiIIerr = FWHMmcdf.loc[i]['LIS Errs']
    HIwidth = FWHMmcdf.loc[i]['FWHM (HI)']
    HIerr = FWHMmcdf.loc[i]['HI Errs']
    style = lbrand.loc[i].T
    c = style['col']
    m = style['sym']
    g = style['siz'] * 8
    if i in [9, 10, 11, 12]:
        g *= 1.3
    elif i in [1, 2, 3, 4]:
        g /= 1.2
    ax.errorbar(HIwidth, SiIIwidth, xerr=HIerr, yerr=SiIIerr, ecolor=c, zorder=0, capthick=1.)
    ax.scatter(HIwidth, SiIIwidth, 
            color=c, marker=m, s=g, edgecolor='k', lw=.5,
            label='{:02d}'.format(i))#.zfill(2)))
    ax.set_xlabel('FWHM (HI) [km s$^{-1}$]', size=9)
    ax.set_ylabel('FWHM (Si II) [km s$^{-1}$]', size=9)
    ax.plot([0, 600], [0, 600], 'k--', lw=.8)
    ax.axis((0, 600, 0, 600))
leg = ax.legend(loc='lower right', ncol=3, scatterpoints=1, markerscale=.7, fontsize=7, 
          fancybox=True, shadow=True, title='LARS ID').draggable()
prop = fm.FontProperties(size=7)
tit = leg.legend.get_title()
tit.set_fontproperties(prop)
leg.legend.get_frame().set_linewidth(0.6)
fig.subplots_adjust(bottom=.12, top=.95, left=.13, right=.9)
    #print thisone, style
fig.savefig('Paper/Figs/FWHMs.pdf', dpi=300)
plt.show()

### Line profile plot with experimental colors

Because the WesAnderson palettes are cool.

In [7]:
import wesanderson as wa
import brewer2mpl as btm
moonrise2 = wa.colors['Royal1']#[1:]
moonrise2 = [farve for farve in wa.colors['Darjeeling2'] if farve != wa.colors['Darjeeling2'][0]]
#moonrise2 = [farve for farve in wa.colors['Darjeeling'] if farve != wa.colors['Darjeeling'][3]]
#moonrise2 = btm.wesanderson.Moonrise3.hex_colors
#moonrise2[1] = btm.wesanderson.Moonrise2.hex_colors[1]

lines = ['Si II 1190', 'Si II 1193', 'Si II 1260', 'Si II 1304']
coldict = dict(zip(lines, moonrise2))
fig, axs = plt.subplots(5, 3, figsize=(7, 5.2), dpi=160,)
axs = axs.flatten()
axs[14].set_visible(False)
#axs[15].set_visible(False)
for i in range(14):
    j = i + 1
    ax = axs[i]
    ax.set_axis_bgcolor('w')
    [k.set_linewidth(.6) for k in ax.spines.itervalues()]
    ar = vflis[j]
    for line in ['Si_II_1190', 'Si_II_1193', 'Si_II_1260', 'Si_II_1304']:
        if not '{}_Flam'.format(line) in ar.columns:
            continue
        index = ar.Velocity
        data = ar['{}_Flam'.format(line)]
        darr = sp.convolve(data, sp.ones(5)/5., mode='same')
        mask = ar['{}_Mask'.format(line)]
        data = sp.ma.masked_array(darr, sp.invert(mask))
        nvrt = sp.ma.masked_array(darr, mask)
        thelabel = r' \rm ' + line.replace('_', ' ')
        ax.plot(index, data, color=coldict[line.replace('_', ' ')], drawstyle='steps-mid', lw=.9, label=thelabel)
        #ax.plot(index, nvrt, color=coldict[line.replace('_', ' ')], drawstyle='steps-mid', ls='--', lw=1.)
        #mp.maskplot(index, data, sp.invert(mask), color=coldict[line.replace('_', ' ')])
    ax.label_outer()
    bbox_props = dict(boxstyle="square, pad=0.4", fc="w", ec='lightgray', lw=.8)
    larsname = r'\rm ' + vflis[j].columns.name
    if larsname[-2:] in LyaEmitters:
        larsname += '*'
    elif larsname[-2:] in LyaAbsorbers:
        larsname += '$\dagger$'
    ax.annotate(larsname, (0.05, 0.8), xycoords='axes fraction', color='k', 
                fontsize='9', family='serif', bbox=bbox_props,)
    ax.axis((-900, 900, -.2, 1.8))
    ax.set_yticks([0., .5,  1., 1.5])
    ax.tick_params(labelsize=8)  # 'medium')
    ax.axhline(y=0, lw=.5, color='k')
    ax.axhline(y=1, lw=.5, color='k', ls='--')
    ax.axvline(x=0, lw=.5, color='k', ls='--')
    if j == 14:
        ax.legend(frameon=False, loc='center', 
                  bbox_to_anchor=(1.5, 0.4), labelspacing=.1, fontsize=11).draggable()
    
cont = fig.add_subplot(111, frameon=False, xticks=[], yticks=[], 
                       xlabel=r'\rm Velocity [km s$^{-1}$]', 
                       ylabel=r'\rm Flux / continuum')
cont.xaxis.label.set_fontsize(9)
cont.yaxis.label.set_fontsize(9)
cont.xaxis.labelpad = 13
cont.yaxis.labelpad = 20
fig.subplots_adjust(left=.07, right=.98, bottom=.09, top=.98, wspace=.02, hspace=.03) 
#fig.savefig('Paper/Figs/liproplot_funky.pdf', dpi=300)
#fig.savefig('LineProfilePlot.png', dpi=300)
fig.set_facecolor('none')
#fig.savefig('EWASS/liproplot_funky.pdf', facecolor='none')#, transparent=True)
plt.show()
#imshow(Uv08, cmap=theBlues, vmin=-2, vmax=2)

NameError: name 'LyaEmitters' is not defined

## Same as above, but now with another panel Showing the avg profile  with the CII and OI profiles

This is the one we are using... I think.

In [1]:
import wesanderson as wa
import brewer2mpl as btm
import matplotlib.gridspec as gs
reload(cosp)
SiIIcolors = btm.wesanderson.Moonrise4.hex_colors[:-1]
SiIIcolors = ['#377eb8', '#4daf4a', '#984ea3', '#ff7f00']
#moonrise2 = [farve for farve in wa.colors['FantasticFox1'] if farve != wa.colors['FantasticFox1'][-1]]
#Moonrise4 = [farve for farve in btm.wesanderson.Moonrise4.hex_colors 
#             if farve != btm.wesanderson.Moonrise4.hex_colors[1]]
OtherColors = ['0.6', '0.25', 'k']# btm.wesanderson.Zissou.hex_colors[:2] + [btm.wesanderson.Darjeeling1.hex_colors[-1]]
colors = SiIIcolors + OtherColors
#moonrise2 = [farve for farve in wa.colors['Darjeeling'] if farve != wa.colors['Darjeeling'][3]]
#moonrise2 = btm.wesanderson.Moonrise3.hex_colors
#moonrise2[1] = btm.wesanderson.Moonrise2.hex_colors[1]
lines = ['Si II 1190', 'Si II 1193', 'Si II 1260', 'Si II 1304',  'C II 1334','O I 1302', 'MeanProfile']
coldict = dict(zip(lines, colors))
#coldict['Si II 1304'] = 'darkcyan'
#coldict['Si II 1304'] = Moonrise4[-1]
fig = plt.figure(figsize=(7, 7), dpi=160,)
#axs = axs.flatten()
#axs[14].set_visible(False)
basegrid = gs.GridSpec(4, 4)
subgrids = {}
LegInfo = {}
#axs[15].set_visible(False)
for i in range(14):
    j = i + 1
    subgrids[j] = gs.GridSpecFromSubplotSpec(2, 1, subplot_spec=basegrid[(i - i%4)/4, i%4], wspace=.2, hspace=0.)
    ax = plt.Subplot(fig, subgrids[j][0])
    ax1 = plt.Subplot(fig, subgrids[j][1], sharex=ax)
    ax.set_axis_bgcolor('w')
    #ax1.set_axis_bgcolor('0.9')
    [k.set_linewidth(.8) for k in ax.spines.itervalues()]
    [k.set_linewidth(.8) for k in ax1.spines.itervalues()]
    ar = vflis[j]
    fig.add_subplot(ax); fig.add_subplot(ax1)

    for line in ['Si_II_1190', 'Si_II_1193', 'Si_II_1260', 'Si_II_1304']:
        if not '{}_Flam'.format(line) in ar.columns:
            continue
        index = ar.Velocity
        data = ar['{}_Flam'.format(line)]
        darr = sp.convolve(data, sp.ones(5)/5., mode='same')
        mask = ar['{}_Mask'.format(line)]
        data = sp.ma.masked_array(darr, sp.invert(mask))
        nvrt = sp.ma.masked_array(darr, mask)
        thelabel = r'\rm ' + line.replace('_', ' ')
        LegInfo[thelabel[4:]] = ax.plot(index, data, color=coldict[line.replace('_', ' ')], 
                                drawstyle='steps-mid', lw=.6, label=thelabel)[0]
        #ax.plot(index, nvrt, color=coldict[line.replace('_', ' ')], drawstyle='steps-mid', ls='--', lw=1.)
        #mp.maskplot(index, data, sp.invert(mask), color=coldict[line.replace('_', ' ')])
    for line in ['Si_II_1304', 'C_II_1334', 'O_I_1302']:
        if (not '{}_Flam'.format(line) in ar.columns) & (line != 'MeanProfile'):
            continue
        index = ar.Velocity
        data = ar['{}_Flam'.format(line)]
        darr = sp.convolve(data, sp.ones(5)/5., mode='same')
        mask = ar['{}_Mask'.format(line)]
        data = sp.ma.masked_array(darr, sp.invert(mask))
        nvrt = sp.ma.masked_array(darr, mask)
        thelabel = r'\rm ' + line.replace('_', ' ')
        LegInfo[thelabel[4:]] = ax1.plot(index, data, color=coldict[line.replace('_', ' ')], 
                                 drawstyle='steps-mid', lw=.6, label=thelabel)[0]
        #ax.plot(index, nvrt, color=coldict[line.replace('_', ' ')], drawstyle='steps-mid', ls='--', lw=1.)
        #mp.maskplot(index, data, sp.invert(mask), color=coldict[line.replace('_', ' ')])
        
    mean_profile = sp.convolve(ar.MeanProfile, sp.ones(5)/5., mode='same')
    ax1.plot(ar.Velocity, mean_profile, 'w-', lw=2.2, drawstyle='steps-mid')
    LegInfo['Average LIS'] = ax1.plot(ar.Velocity, mean_profile, 'k-', lw=1.2, drawstyle='steps-mid')[0]

    
    bbox_props = dict(boxstyle="square, pad=0.4", fc="w", ec='lightgray', lw=.8)
    larsname = r'\rm ' + vflis[j].columns.name
    if larsname[-2:] in LyaEmitters:
        larsname += '*'
    elif larsname[-2:] in LyaAbsorbers:
        larsname += '$\dagger$'
    ax.annotate(larsname, (0.05, 0.8), xycoords='axes fraction', color='k', 
                fontsize='9', family='serif')#, bbox=bbox_props,)
    ax.axis((-900, 900, -.2, 2.))
    ax1.axis((-900, 900, -.2, 2.))
    ax.set_yticks([0., .5,  1., 1.5])
    ax1.set_yticks([0., .5,  1., 1.5])
    ax.tick_params(labelsize=8, length=2)  # 'medium')
    ax1.tick_params(labelsize=8, length=2)  # 'medium')
    if i%4 != 0:
        ax.tick_params(labelleft='off')
        ax1.tick_params(labelleft='off')
    ax.axhline(y=0, lw=.5, color='k', ls='--', zorder=0)
    ax.axhline(y=1, lw=.5, color='k', ls=':', zorder=0)
    ax.axvline(x=0, lw=.5, color='k', ls=':', zorder=0)
    ax1.axhline(y=0, lw=.5, color='k', ls='--',zorder=0)
    ax1.axhline(y=1, lw=.5, color='k', ls=':', zorder=0)
    ax1.axvline(x=0, lw=.5, color='k', ls=':', zorder=0)
    #if j == 14:
    #    ax.legend(frameon=False, loc='center', 
    #              bbox_to_anchor=(1.5, 0.4), labelspacing=.1, fontsize=11).draggable()
leglabels = ['Si II 1190', 'Si II 1193', 'Si II 1260', 'Si II 1304', 'O I 1302', 'C II 1334', 'Average LIS']
leg = plt.figlegend([LegInfo[key] for key in leglabels], leglabels, 'lower right', frameon=False, 
                    ncol=2, handletextpad=.4, labelspacing=.3, markerscale=8., handlelength=1.5, 
                    columnspacing=.6, fontsize=11, bbox_to_anchor=(.94, .07))
cont = fig.add_subplot(111, frameon=False, xticks=[], yticks=[], 
                       xlabel=r'\rm Velocity [km s$^{-1}$]', 
                       ylabel=r'\rm Flux / continuum')
cont.xaxis.label.set_fontsize(9)
cont.yaxis.label.set_fontsize(9)
cont.xaxis.labelpad = 13
cont.yaxis.labelpad = 20
fig.subplots_adjust(left=.07, right=.98, bottom=.05, top=.98, wspace=.03, hspace=.03) 
#fig.savefig('Paper/Figs/liproplot_funky.pdf', dpi=300)
#fig.savefig('LineProfilePlot.png', dpi=300)
fig.set_facecolor('none')
#fig.savefig('EWASS/liproplot_funky.pdf', facecolor='none')#, transparent=True)
plt.show()
#imshow(Uv08, cmap=theBlues, vmin=-2, vmax=2)

NameError: name 'cosp' is not defined

In [59]:
LegInfo

{' \\rm C II 1334': <matplotlib.lines.Line2D at 0x7f6916ca8ed0>,
 ' \\rm O I 1302': <matplotlib.lines.Line2D at 0x7f69143d3dd0>,
 ' \\rm Si II 1190': <matplotlib.lines.Line2D at 0x7f6915e89350>,
 ' \\rm Si II 1193': <matplotlib.lines.Line2D at 0x7f6915e89510>,
 ' \\rm Si II 1260': <matplotlib.lines.Line2D at 0x7f6915e89bd0>,
 ' \\rm Si II 1304': <matplotlib.lines.Line2D at 0x7f6916ca8890>,
 'Average LIS': <matplotlib.lines.Line2D at 0x7f69143ca250>}

### Plot showing how we do the coverfing fraction fits

In [16]:
import wesanderson as wa
moonrise2 = wa.colors['Royal1']#[1:]
moonrise2 = [farve for farve in wa.colors['Darjeeling2'] if farve != wa.colors['Darjeeling2'][0]]
lines = ['Si II 1190', 'Si II 1193', 'Si II 1260', 'Si II 1304']
coldict = dict(zip(lines, moonrise2))
fig, ax = plt.subplots(1)
fig.set_size_inches(3.5, 3)
fig.dpi = 160
ar = vflis[1]
Vel = ar.Velocity[289]
for line in ['Si_II_1190', 'Si_II_1193', 'Si_II_1260', 'Si_II_1304']:
    if not '{}_Flam'.format(line) in ar.columns:
        continue
    index = ar.Velocity
    data = ar['{}_Flam'.format(line)]
    darr = sp.convolve(data, sp.ones(5)/5., mode='same')
    mask = ar['{}_Mask'.format(line)]
    data = sp.ma.masked_array(darr, sp.invert(mask))
    nvrt = sp.ma.masked_array(darr, mask)
    thelabel = line.replace('_', ' ')
    ax.plot(index, data, color=coldict[line.replace('_', ' ')], 
            drawstyle='steps-mid', lw=1, label=line.replace('_', ' '))
    ax.plot(Vel, data[289], 'ro', markersize=7, mec='lightgray')
    #ax.plot(index, nvrt, color=coldict[line.replace('_', ' ')], 
    #        drawstyle='steps-mid', ls='--', lw=1.)
    #mp.maskplot(index, data, sp.invert(mask), color=coldict[line.replace('_', ' ')])
ax.label_outer()
#ax.tick_params(labelbottom='off', labelleft='off')
ax.annotate(ar.columns.name, (0.05, 0.9), xycoords='axes fraction', color='k', 
            family='serif', fontsize=10)
ax.axis((-620, 620, -.0, 1.4))
#ax.set_yticks([0., .5,  1., 1.5])
#ax.tick_params(labelsize='medium')
ax.axhline(y=0, lw=.5, color='k', zorder=0)
ax.axhline(y=1, lw=.5, color='k', ls='--', zorder=0)
ax.axvline(x=0, lw=.5, color='k', ls='--', zorder=0)

ax.legend(loc='lower right', fancybox=True, shadow=True, labelspacing=.2, fontsize=9).draggable()

ax.tick_params(labelsize=8, axis='both')
#fig.savefig('EWASS/howtofit.pdf', facecolor='none')
plt.show()

IndexError: too many indices for array

In [19]:
data

masked_array(data = [0.5781950804261959 0.7982892744688933 1.014788669265604 1.040076205306019
 1.0497286812749846 1.0467350059751142 1.0135887873560265
 1.0063791206557933 0.9953591985816364 0.9756727476266039
 0.9747317205329621 0.9999819020328643 1.0083170280289564 1.049187384192977
 1.0778425133470324 1.0817624564621668 1.085292847653675 1.0730808489381873
 1.0565500487334947 1.0468836394660765 1.1026984583638442
 1.0965672370382271 1.0739418869235164 1.0603348837129736
 1.0440513499058426 1.0049204847628925 1.0130922669361346
 1.0516433732215775 1.0573016927170524 1.06066346371959 1.0496687929303123
 1.0392187751919286 1.0216096690186338 1.0230146160574878
 1.0383198935795528 1.0383680138739835 1.0177343509354992 1.035171534545419
 1.0363728966294203 1.0249071243708314 1.0180527389120928
 1.0137081709186355 0.962716004247568 0.9340774295293258 0.927577120433621
 0.9241630200296183 0.9385672672051545 0.955306365182466 0.9571159800146718
 0.9354612607502317 0.9093480479505612 0.9147

## Dynamic Velocities, now with niceness

In [25]:
#import cospaper as cosp
reload(cosp)
reload(dv)
import wesanderson as wa
wes = [farve for farve in wa.colors['Darjeeling2'] if farve != wa.colors['Darjeeling2'][0]]
#import DynVel as dv
%pylab
vflis = cosp.load_dfdict('vflis.pickle')
#vflis[1]
dv.dynvel(vflis[1])
fig, ax = gcf(), gca()
polything = ax.collections[0]
polything.set_color(wes[0])
polything.set_alpha(0.6)
draw()
savefig('Paper/Figs/DynVel-SiII-LARS_01.pdf', facecolor='none')

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


Using default event loop until function specific to this GUI is implemented


In [23]:
fig, ax = gcf(), gca()
polything = ax.collections[0]
polything.set_color(wes[0])
polything.set_alpha(0.6)
leg = ax.get_legend()

draw()

### Dynamic velocities, now with uncertainties through Monte Carlo

In [13]:
import cospaper as cosp
import DynVel as dv
import numpy as np
import scipy as sp
reload(dv)
rc('text', usetex=True)
vfd = cosp.load_dfdict('vflis.pickle')

In [93]:
def randomize(indata, rng=None):
    """Creates a fake dataset by adding a random sample from a gaussian 
    modelled over the actual error distribution to the original data.
    Use result for monte carlo error determination.
    rng: array-like
         a list,  tuple or array of (min, max) of the velocity interval 
         in which to sample the errors.
    """
    # For convenience
    data = indata.MeanProfile
    errs = indata.MeanErrors
    if rng is None:
        rng = [indata.Velocity.min(), indata.Velocity.max()]
    errs = errs[indata.Velocity.between(rng[0], rng[1])]
    # Find distribution of the errors.
    errmean = errs.mean()
    errdev = errs.std()
    # Draw random sample from error distribution
    newerrs = sp.random.normal(errmean, errdev, data.shape[0])
    # Draw random sequence of +/-
    signs = sp.around(sp.random.random(data.shape[0]))
    signs[signs==0] = -1
    # Add or subtract,  randomly chosen, the errrors from/to the measurements
    newerrs *= signs
    indata.MeanErrors = newerrs
    indata.MeanProfile += newerrs
    #indata = indata[indata.Velocity.between(rng[0], rng[1])]
    #print rng
    return indata



def get_ranges(indata):
    spedata = indata.MeanProfile
    wl = indata.Velocity
    fig = plt.figure()  # figsize=(3.4, 3.4), dpi=160)
    ax = plt.subplot(111)
    exes = []
    plt.plot(wl, spedata, 'black', drawstyle='steps-mid')
    plt.axis((-800, 800, -.1, 1.4))
    plt.axhline(y=1., linestyle='--', color='k')
    plt.annotate(r'''Mark piece of left continuum, beginning and end of line, \\
                 and piece of right continuum, 6 clicks in total''',
                 (.5, .9), xycoords='axes fraction',
                 ha='center')
    #cid = fig.canvas.mpl_connect('button_press_event', onclick)
    #plt.show()
    exes = plt.ginput(6)
    plt.close()
    #import pdb; pdb.set_trace() ### XXX BREAKPOINT
    #plt.show()
    # print exes
    leftexes = np.asarray(exes[0:2])[:, 0]
    riteexes = np.asarray(exes[4:6])[:, 0]
    # print 'riteexes', riteexes
    lineexes = np.asarray(exes[2:4])[:, 0]
    exesarr = np.array([leftexes, lineexes, riteexes])
    return exesarr
    
def DVMC(indata, ranges=None, iterations=500):
    import sys
    if ranges is not None:
        rng = [ranges[1, 0], ranges[1, 1]]
    else:
        ranges = get_ranges(indata)
        rng = [ranges[1, 0], ranges[1, 1]]
    #data = indata.MeanProfile
    #errs = indata.MeanErrors
    # Real values:
    dvdict = dv.dynvel(indata, ranges=ranges, plot=False)
    # Errors:
    errsdict = {i: [] for i in dvdict.keys()}
    count = 0
    for j in range(iterations):
        sys.stdout.write("\rIteration:  {}, II0min = {}".format(j, dvdict['I/I0_min']))
        sys.stdout.flush()
        mcdata = indata.copy()
        mcdata = randomize(mcdata, rng=rng)
        # Because some configurations with large errors break down also make the code break down.
        # This is kinda cheating, but shouldn't be too much. Only discards configurations oof too 
        try: 
            mcdict = dv.dynvel(mcdata, ranges=ranges, plot=False)
        except:
            print('Iteration {} failed'.format(j))
            continue
        count += 1
        #print('{}'.format(mcdict))
        for j in mcdict.keys():
            errsdict[j].append(mcdict[j])
    errframe = pd.DataFrame.from_dict(errsdict)
    for quant in errframe.columns:
        err = errframe[quant].std()
        dvdict[quant+'_Std'] = err
    dvdict['count'] = count
    return dvdict, errframe#, errsdict

In [94]:
reload(cosp)
reload(dv)
velsdf = {}
#ranges = get_ranges(vfd[6])
for galaxy in vfd.keys():
    ranges = get_ranges(vfd[galaxy])
    itr = 500
    dvdict, errframe = DVMC(vfd[galaxy], ranges=ranges, iterations=itr)
    velsdf[galaxy] = dvdict
#print velsdf
#dvframe = pd.DataFrame.from_dict(dvdict)

Iteration:  499, II0min = 0.301932528142

Using default event loop until function specific to this GUI is implemented
Using default event loop until function specific to this GUI is implemented


Iteration:  499, II0min = 0.0324748577685

Using default event loop until function specific to this GUI is implemented
Using default event loop until function specific to this GUI is implemented


Iteration:  499, II0min = 0.186758030184

...

Iteration:  499, II0min = -0.039830215164

Using default event loop until function specific to this GUI is implemented
Using default event loop until function specific to this GUI is implemented


Iteration:  499, II0min = 0.241086641036

Using default event loop until function specific to this GUI is implemented
Using default event loop until function specific to this GUI is implemented


Iteration:  499, II0min = 0.596544940618

In [95]:
velsframe = pd.DataFrame.from_dict(velsdf).T
velsframe['Fcmax'] = 1 - velsframe['I/I0_min']
velsframe['Fcmax_Std'] = velsframe['I/I0_min_Std']
#velsframe = velsframe.drop(['I/I0_0', 'I/I0_0_Stddev', 'I/I0_min', 'I/I0_min_Stddev'], axis=1)
velsframe.delta_v = velsframe.delta_v.map(lambda x: sp.around(x))
velsframe.delta_v_Std = velsframe.delta_v_Std.map(lambda x: sp.around(x))
velsframe.v_95 = velsframe.v_95.map(lambda x: sp.around(x))
velsframe.v_95_Std = velsframe.v_95_Std.map(lambda x: sp.around(x))
velsframe.v_int = velsframe.v_int.map(lambda x: sp.around(x))
velsframe.v_int_Std = velsframe.v_int_Std.map(lambda x: sp.around(x))
velsframe.v_min = velsframe.v_min.map(lambda x: sp.around(x))
velsframe.v_min_Std = velsframe.v_min_Std.map(lambda x: sp.around(x))
velsframe.Fcmax = velsframe.Fcmax.map(lambda x: sp.around(x, 2))
velsframe.Fcmax_Std = velsframe.Fcmax_Std.map(lambda x: sp.around(x, 2))
velsframe['I/I0_min'] = velsframe['I/I0_min'].map(lambda x: sp.around(x, 2))
velsframe['I/I0_min_Std'] = velsframe['I/I0_min_Std'].map(lambda x: sp.around(x, 2))
velsframe['I/I0_0'] = velsframe['I/I0_0'].map(lambda x: sp.around(x, 2))
velsframe['I/I0_0_Std'] = velsframe['I/I0_0_Std'].map(lambda x: sp.around(x, 2))
velsframe.columns.name = 'LARS ID'
velsframe.to_csv('velsframe.csv')
velsframe

LARS ID,I/I0_0,I/I0_0_Std,I/I0_min,I/I0_min_Std,count,delta_v,delta_v_Std,v_95,v_95_Std,v_int,v_int_Std,v_min,v_min_Std,Fcmax,Fcmax_Std
1,0.45,0.12,0.30,0.03,500,381,34,-314,24,-100,12,-65,21,0.70,0.03
2,0.19,0.32,0.14,0.07,500,351,102,-258,80,-48,33,-24,25,0.86,0.07
3,0.13,0.23,0.03,0.05,500,628,51,-462,42,-118,73,-112,53,0.97,0.05
4,0.05,0.27,0.01,0.07,500,365,61,-261,46,-31,152,-13,31,0.99,0.07
5,0.52,0.11,0.19,0.03,500,466,47,-390,22,-139,14,-84,24,0.81,0.03
6,0.16,0.32,0.16,0.06,500,539,43,-229,38,68,28,4,93,0.84,0.06
7,0.22,0.14,0.20,0.03,500,406,68,-282,63,-39,18,9,35,0.80,0.03
8,0.56,0.30,0.02,0.07,500,537,85,-471,70,-242,55,-225,53,0.98,0.07
9,0.02,0.16,0.00,0.05,500,503,34,-263,27,40,140,82,50,1.00,0.05
10,-0.03,0.33,-0.03,0.08,500,470,51,-273,38,-27,187,-3,49,1.03,0.08


In [ ]:
print velsframe

In [35]:
dvdict

{'I/I0_0': 0.51656868882496609,
 'I/I0_0_Stddev': 0.11231222830156,
 'I/I0_min': 0.18675803018400608,
 'I/I0_min_Stddev': 0.027877251175373395,
 'delta_v': 465.99139854912562,
 'delta_v_Stddev': 45.276968125313644,
 'v_95': -389.55248740752478,
 'v_95_Stddev': 22.904050384625503,
 'v_int': -138.76182482803819,
 'v_int_Stddev': 13.965755399826968,
 'v_min': -83.745632109661074,
 'v_min_Stddev': 21.018453506829189}

In [21]:
print dvdict['v_int'], dvdict['v_int_Stddev']
print errframe['v_int'].mean()# + vfd[1].MeanErrors.mean()
print errframe['v_int'].std()
errframe['v_int'].hist()
plt.show()

-100.228089262 13.0811376314
-105.119280801
13.0811376314


## Table of velocity widths and covering fractions 


In [14]:
velwidths = pd.read_csv('velwidths.csv', index_col=0)
velwidths['$f_c (v)$'] = 1. - velwidths['I/I0_min']
velwidths = velwidths[['delta_v', 'v_int', '$f_c (v)$']]
velwidths.columns = ['$\Delta v$', '$v_{\rm int}$', '$f_{c, \mathrm{max}} (v)$']
velwidths.columns.name = 'LARS ID'
velwidths.index = velwidths.index.map(lambda x: x[-2:])
velwidths

LARS ID,$\Delta v$,$v_{ m int}$,"$f_{c, \mathrm{max}} (v)$"
01,381,-100,0.70
02,351,-48,0.86
03,628,-117,0.97
04,365,-31,0.99
05,466,-138,0.81
06,553,68,0.84
07,392,-38,0.80
08,522,-240,0.98
09,503,40,1.00
10,484,-27,1.03


In [111]:
bigone = velwidths.join?
FWHMs = FWHMdf['FWHM']
deltav = velwidths['$\Delta v$']

#plt.plot(deltav, FWHMs, 'ko')
#plt.show()
bigone

,$\Delta v$,$v_{ m int}$,"$f_{c, \mathrm{max}} (v)$",FWHM,Maxvel,Minvel
01,381,-100,0.70,297.099638,55.136901,-241.962737
02,351,-48,0.86,208.660664,49.789394,-158.871270
03,628,-117,0.97,480.686897,117.026383,-363.660515
04,365,-31,0.99,263.645833,90.922522,-172.723311
05,466,-138,0.81,365.443986,18.829530,-346.614456
06,553,68,0.84,543.567037,316.843615,-226.723422
07,392,-38,0.80,240.055108,85.863325,-154.191783
08,522,-240,0.98,388.884756,-46.975936,-435.860691
09,503,40,1.00,417.898750,195.955067,-221.943683
10,484,-27,1.03,372.947561,154.094738,-218.852823


In [ ]:
bigone = velwidths.join

In [36]:
#FWHMdf.index = FWHMdf.index.map(lambda x: '{:02d}'.format(x))
bigone = velwidths.join(FWHMdf)
bigone.corr()

,$\Delta v$,$v_{ m int}$,"$f_{c, \mathrm{max}} (v)$",FWHM,Maxvel,Minvel
$\Delta v$,1.000000,-0.034349,0.105089,0.905231,0.363754,-0.517040
$v_{\rm int}$,-0.034349,1.000000,0.264660,0.080008,0.880355,0.833863
"$f_{c, \mathrm{max}} (v)$",0.105089,0.264660,1.000000,0.026459,0.131472,0.110194
FWHM,0.905231,0.080008,0.026459,1.000000,0.513075,-0.455817
Maxvel,0.363754,0.880355,0.131472,0.513075,1.000000,0.530121
Minvel,-0.517040,0.833863,0.110194,-0.455817,0.530121,1.000000


In [51]:
#print FWHMs.index
#FWHMs.index = FWHMs.index.map(lambda x: '{:02d}'.format(x))
print FWHMs.index
print deltav.index
model = pd.ols(y=FWHMs, x=deltav)
model
#from sklearn.linear_model import LinearRegression
#from sklearn.metrics import r2_score
#lr = LinearRegression()
#lr.fit(FWHMs, deltav)
#b_0 = lr.intercept_
#coeff = lr.coef_

Index([u'01', u'02', u'03', u'04', u'05', u'06', u'07', u'08', u'09', u'10', u'11', u'12', u'13', u'14'], dtype='object')
Index([u'01', u'02', u'03', u'04', u'05', u'06', u'07', u'08', u'09', u'10', u'11', u'12', u'13', u'14'], dtype='object')



-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <x> + <intercept>

Number of Observations:         14
Number of Degrees of Freedom:   2

R-squared:         0.8194
Adj R-squared:     0.8044

Rmse:             41.0054

F-stat (1, 12):    54.4614, p-value:     0.0000

Degrees of Freedom: model 1, resid 12

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
             x     1.0845     0.1470       7.38     0.0000     0.7964     1.3725
     intercept  -135.5023    69.6203      -1.95     0.0754  -271.9580     0.9534
---------------------------------End of Summary---------------------------------

In [71]:
#print model.y_fitted
#print model.x
plt.plot(model.x.x.values, model.y_fitted.values, 'k')
plt.plot(deltav, FWHMs, 'ko')
plt.show()

In [91]:
from scipy.stats import linregress as lrg
slope, intercapt, r_val, p_val, std_err = lrg(deltav, FWHMs)
print r_val, p_val

0.905231492152 8.50362526598e-06


In [88]:
print deltav.values
print FWHMdf.values

[ 381.  351.  628.  365.  466.  553.  392.  522.  503.  484.  433.  503.
  484.  485.]
[[ 297.09963819   55.13690145 -241.96273674]
 [ 208.66066358   49.78939377 -158.8712698 ]
 [ 480.68689723  117.02638256 -363.66051467]
 [ 263.6458325    90.92252165 -172.72331086]
 [ 365.44398614   18.82953008 -346.61445606]
 [ 543.567037    316.84361489 -226.7234221 ]
 [ 240.05510767   85.86332463 -154.19178304]
 [ 388.8847556   -46.97593564 -435.86069124]
 [ 417.89875008  195.95506721 -221.94368287]
 [ 372.94756082  154.09473794 -218.85282288]
 [ 377.56551969   -8.95956772 -386.52508741]
 [ 431.74632628  204.36439751 -227.38192877]
 [ 420.96639996  116.40207544 -304.56432451]
 [ 397.09014977   11.9021397  -385.18801007]]


In [115]:
print deltav
print deltav[deltav.argsort().values]

01    381
02    351
03    628
04    365
05    466
06    553
07    392
08    522
09    503
10    484
11    433
12    503
13    484
14    485
Name: $\Delta v$, dtype: float64
02    351
04    365
01    381
07    392
11    433
05    466
10    484
13    484
14    485
09    503
12    503
08    522
06    553
03    628
Name: $\Delta v$, dtype: float64


In [116]:
import numpy as np
import math
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.sandbox.regression.predstd import wls_prediction_std

#measurements genre
nmuestra = 100

re = sm.OLS(deltav[deltav.argsort().values], FWHMs[deltav.argsort().values]).fit()
print re.summary()    #print the result type Stata

prstd, iv_l, iv_u = wls_prediction_std(re)

                            OLS Regression Results                            
Dep. Variable:             $\Delta v$   R-squared:                       0.987
Model:                            OLS   Adj. R-squared:                  0.986
Method:                 Least Squares   F-statistic:                     989.8
Date:                Sat, 10 May 2014   Prob (F-statistic):           1.18e-13
Time:                        03:59:06   Log-Likelihood:                -75.696
No. Observations:                  14   AIC:                             153.4
Df Residuals:                      13   BIC:                             154.0
Df Model:                           1                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
FWHM           1.2307      0.039     31.461      0.000         1.146     1.315
Omnibus:                        1.921   Durbin-Watso

kurtosistest only valid for n>=20 ... continuing anyway, n=14


In [117]:
from statsmodels.stats.outliers_influence import summary_table

st, data, ss2 = summary_table(re, alpha=0.05)

fittedvalues = data[:,2]
predict_mean_se  = data[:,3]
predict_mean_ci_low, predict_mean_ci_upp = data[:,4:6].T
predict_ci_low, predict_ci_upp = data[:,6:8].T

# check we got the right things
print np.max(np.abs(re.fittedvalues - fittedvalues))
print np.max(np.abs(iv_l - predict_ci_low))
print np.max(np.abs(iv_u - predict_ci_upp))

plt.plot(deltav, FWHMs, 'o')
plt.plot(deltav, fittedvalues, '-', lw=2)
plt.plot(deltav, predict_ci_low, 'r--', lw=2)
plt.plot(deltav, predict_ci_upp, 'r--', lw=2)
plt.plot(deltav, predict_mean_ci_low, 'r--', lw=2)
plt.plot(deltav, predict_mean_ci_upp, 'r--', lw=2)
plt.show()

0.0
0.0
0.0


In [36]:
ar.Velocity[285: 295]

285   -64.915695
286   -50.262979
287   -35.610263
288   -20.957546
289    -6.304830
290     8.347886
291    23.000602
292    37.653319
293    52.306035
294    66.958751
Name: Velocity, dtype: float64

In [37]:
ar.Velocity[289].repeat(4)

array([-6.30483014, -6.30483014, -6.30483014, -6.30483014])

In [64]:
cont.xaxis.labelpad = 20

In [273]:
reload(cosp)
cosp.make_line_profile_plots(indict=dfd, lines='lis')
plt.show()

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
0.0278943985294
0.0278943985294 normalize_lines
0.0278943985294 normalize_line() 1
Normalised version of line Si II 1190 already present in the dataset LARS 01
0.0278943985294 normalize_line() 1
Normalised version of line Si II 1193 already present in the dataset LARS 01
0.0278943985294 normalize_line() 1
Normalised version of line Si II 1304 already present in the dataset LARS 01
0.0278943985294 normalize_line() 1
Normalised version of line Si II 1260 already present in the dataset LARS 01
0.0297475784535
0.0297475784535 normalize_lines
0.0297475784535 normalize_line() 1
Normalised version of line Si II 1190 already present in the dataset LARS 02
0.0297475784535 normalize_line() 1
Normalised version of line Si II 1193 already present in the dataset LARS 02
0.0297475784535 normalize_line() 1
Normalised version of line Si II 1304 already present in the dataset LARS 02
0.0297475784535 normalize_line() 1
Normalised version of line Si II 1260 alr

In [25]:
fig2 = plt.figure()
for i in thelist:
    fig2.axes.append(i)
plt.show()

In [17]:
AA = sp1[5]
AA.colNum

2

In [12]:
plt.subplots?

# Wes Anderson color maps from `brewer2mpl`

...basically scissored from [here]

[here]: http://nbviewer.ipython.org/github/jiffyclub/brewer2mpl/blob/master/demo/brewer2mpl_maps.ipynb

In [70]:
from brewer2mpl import qualitative, sequential, diverging, wesanderson
from brewer2mpl.brewer2mpl import _ColorMap

In [71]:
def display_maps(module):
    for k in dir(module):
        thing = getattr(module, k)
        if isinstance(thing, dict) and 'max' in thing:
            thing = thing['max']
        if isinstance(thing, _ColorMap):
            print(k)
            thing.show_as_blocks()

In [72]:
display_maps(wesanderson)

Cavalcanti


,,,,


Chevalier


,,,


Darjeeling1


...

,,,,


Royal1


,,,


Zissou


,,,,
